In [1]:
import pandas as pd

## Getting tracts with their respective coordinates

In [2]:
tracts = pd.read_csv('./Mapping/tract_coordinates.csv')
tracts.head()

,tractid,latitude,longitude
0,6075010100,37.808420,-122.408579
1,6075010200,37.805861,-122.422925
2,6075010300,37.801620,-122.415681
3,6075010400,37.802273,-122.407558
4,6075010500,37.802684,-122.399059


## Fetching restaurant data

In [3]:
data_path = './Restaurants/SFBusinesses/'
files = [data_path+f for f in ['businesses.csv', 'inspections.csv','violations.csv']]
# 'businesses_partially_completed.csv', 'tract_coordinates.csv']

In [4]:
# bus = pd.read_csv(files[0])
# bus.head()

In [5]:
# # Fill in missing latitudes and longitudes
# from geopy.geocoders import Nominatim
# geolocator = Nominatim()
# bus_filled = pd.read_csv(files[0]).drop('phone_number', axis=1)
# j=0
# for i in bus.index:
#     restau = bus.ix[i]
#     if not restau['latitude'] > 0:
#         print j
#         j+=1
#         try:
#             location = geolocator.geocode(restau['address']+' San Francisco, California')
#             bus_filled.loc[i, 'latitude'] = location.latitude
#             bus_filled.loc[i, 'longitude'] = location.longitude
#         except:
#             print 'problem'

In [6]:
# bus_filled.to_csv(data_path+'businesses_partially_completed.csv', index=False)

In [7]:
bus = pd.read_csv(data_path+'businesses_partially_completed.csv')
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641


In [8]:
bus = bus.drop(['city', 'state'], axis=1)
bus.head()

,business_id,name,address,postal_code,latitude,longitude
0,10,TIRAMISU KITCHEN,033 BELDEN PL,94104,37.791116,-122.403816
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",94109,37.786848,-122.421547
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",94104,37.792888,-122.403135
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,94133,37.807155,-122.419004
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,94110,37.747114,-122.413641


In [9]:
inspect = pd.read_csv(files[1])
inspect.head()

,business_id,score,date,type
0,10,94,20140729,routine
1,10,92,20140114,routine
2,19,94,20141110,routine
3,19,94,20140214,routine
4,19,96,20130904,routine


In [10]:
# Keep most recent inspection only
most_recent_scores = inspect.groupby('business_id').first().drop(['type', 'date'], axis=1)
most_recent_scores.head()

,score
business_id,
10,94
19,94
24,96
31,98
45,78


In [11]:
bus_scores = pd.merge(bus, most_recent_scores, left_on='business_id', right_index=True, how='outer')

In [12]:
bus_scores.count()

business_id    6162
name           6162
address        6161
postal_code    5922
latitude       6162
longitude      6162
score          5515
dtype: int64

In [13]:
bus_scores = bus_scores.dropna()
bus_scores.head()

,business_id,name,address,postal_code,latitude,longitude,score
0,10,TIRAMISU KITCHEN,033 BELDEN PL,94104,37.791116,-122.403816,94.0
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",94109,37.786848,-122.421547,94.0
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",94104,37.792888,-122.403135,96.0
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,94133,37.807155,-122.419004,98.0
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,94110,37.747114,-122.413641,78.0


## Make the index

In [14]:
from density_estimation import point_density, average_score

In [15]:
scores = tracts.apply(lambda x: average_score((x['latitude'], x['longitude']), 
                                              (bus_scores['latitude'], bus_scores['longitude']), 
                                              bus_scores['score'], kernel='gaussian', bw=0.03), axis=1)

In [16]:
restaurants_close = tracts.apply(lambda x: point_density((x['latitude'], x['longitude']), 
                                                         (bus_scores['latitude'], bus_scores['longitude']),
                                                         bw=0.03), axis=1)
restaurants_close *= 100. / restaurants_close.max()

In [17]:
restaurants_close.ix[:20]

0      43.444391
1      29.997232
2      61.812202
3      70.648950
4      61.752785
5      81.971559
6      82.749776
7      81.148118
8      58.851740
9      69.060806
10     75.702608
11     91.604636
12     93.913841
13    100.000000
14     98.129580
15     91.335391
16     98.715050
17     78.056039
18     96.145835
19     83.564982
20     72.744452
dtype: float64

In [18]:
feat = pd.read_csv('features.csv')
feat.head()

,tractid,latitude,longitude,crime_index,rest_proximity,rest_score
0,6075010100,37.808420,-122.408579,10.896802,43.444391,90.601886
1,6075010200,37.805861,-122.422925,11.077546,29.997232,90.295370
2,6075010300,37.801620,-122.415681,21.613337,61.812202,90.457677
3,6075010400,37.802273,-122.407558,21.119650,70.648950,90.652702
4,6075010500,37.802684,-122.399059,12.413430,61.752785,90.873398


In [19]:
feat['rest_proximity'] = restaurants_close
feat['rest_score'] = scores
feat.head()

,tractid,latitude,longitude,crime_index,rest_proximity,rest_score
0,6075010100,37.808420,-122.408579,10.896802,43.444391,90.601886
1,6075010200,37.805861,-122.422925,11.077546,29.997232,90.295370
2,6075010300,37.801620,-122.415681,21.613337,61.812202,90.457677
3,6075010400,37.802273,-122.407558,21.119650,70.648950,90.652702
4,6075010500,37.802684,-122.399059,12.413430,61.752785,90.873398


In [20]:
feat.to_csv('features.csv', index=False)